# Computing Running time, ARI, NMI through un-normalized Laplacian matrix for our understanding

In [1]:
from sklearn.datasets import fetch_openml
from sklearn.neighbors import kneighbors_graph
from sklearn.cluster import SpectralClustering, KMeans
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt
import time

In [2]:
def Lloyds_algo(data, num_clusters, tolerance, max_iterations=1000):
  data= np.array(data)
  random_choice = np.random.choice(data.shape[0], num_clusters)
  x = data[random_choice]
  prev_distortion = None
  iterations = 0

  for i in range(max_iterations):
    diff= data - x.reshape(x.shape[0],1,x.shape[1])
    diff_squared = diff **2
    diff_squared_sum = np.sum(diff_squared, axis=2)
    euclid= np.sqrt(diff_squared_sum)
    p_ij = np.argmin(euclid, axis=0)

    x_new = np.array([data[p_ij == i].mean(axis=0) for i in range(num_clusters)])

    distortion = 0
    for j in range(num_clusters):
      distortion += np.sum((data[p_ij == j] - x[j]) ** 2) #/data1[p_ij == j].shape[0] #np.sum(data1[p_ij == j] ** 2)

    if np.linalg.norm(x_new - x) < tolerance or (prev_distortion is not None and np.linalg.norm(distortion - prev_distortion) < tolerance): #tolerance:
      break

    x = x_new
    prev_distortion = distortion
    iterations+=1

  return x, p_ij, distortion, num_clusters

In [3]:
def kmeans_algorithm(data, num_clusters, tolerance, max_iterations=1000):
    kmeans = KMeans(n_clusters=num_clusters, init='random', n_init=1, max_iter=max_iterations, tol=tolerance)
    kmeans.fit(data)
    centroids = kmeans.cluster_centers_
    labels = kmeans.labels_
    distortion = kmeans.inertia_
    running_time = kmeans.n_iter_

    return centroids, labels, distortion, num_clusters

In [3]:
def Spectral_clustering_knn_connectivity(data,K,k):
  A=kneighbors_graph(data,k, mode='connectivity')
  A=A.toarray()
  d_mat_k=np.zeros_like(A)
  for i in range(len(A)):
    d_mat_k[i][i]=np.sum(A[i])
  L_mat_k = d_mat_k - A
  eigenvalues_k, eigenvectors_k = np.linalg.eig(L_mat_k)
  sorted_indices_k = np.argsort(eigenvalues_k)#[::-1]
  sorted_eigenvalues_k = eigenvalues_k[sorted_indices_k]
  sorted_eigenvectors_k = eigenvectors_k[:, sorted_indices_k]
  first_k_eigenvectors_k = sorted_eigenvectors_k[:,:K]
  # first_k_eigenvectors_k
  # sorted_eigenvectors_k
  x, p_ij, distortion, num_clusters = Lloyds_algo(first_k_eigenvectors_k,K,1e-5)
  # x, p_ij, distortion, num_clusters = kmeans_algorithm(first_k_eigenvectors_k,K,1e-5)

  return x, p_ij, distortion, num_clusters, A, first_k_eigenvectors_k

In [4]:
def Spectral_clustering_knn_distance(data,K,k):
  A=kneighbors_graph(data,k, mode='distance')
  A=A.toarray()
  d_mat_k=np.zeros_like(A)
  for i in range(len(A)):
    d_mat_k[i][i]=np.sum(A[i])
  L_mat_k = d_mat_k - A
  eigenvalues_k, eigenvectors_k = np.linalg.eig(L_mat_k)
  sorted_indices_k = np.argsort(eigenvalues_k)#[::-1]
  sorted_eigenvalues_k = eigenvalues_k[sorted_indices_k]
  sorted_eigenvectors_k = eigenvectors_k[:, sorted_indices_k]
  first_k_eigenvectors_k = sorted_eigenvectors_k[:,:K]
  # first_k_eigenvectors_k
  # sorted_eigenvectors_k
  x, p_ij, distortion, num_clusters = Lloyds_algo(first_k_eigenvectors_k,K,1e-5)
  # x, p_ij, distortion, num_clusters = kmeans_algorithm(first_k_eigenvectors_k,K,1e-5)

  return x, p_ij, distortion, num_clusters, A, first_k_eigenvectors_k

In [5]:
# Load the HAR dataset (assuming it's available in scikit-learn)
har = fetch_openml(name='har')

# Extract features (X) and labels (y)
X = har.data
y = har.target

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [6]:
start_conn = time.time()
x_knn_conn, p_ij_knn_conn, distortion_knn_conn, num_clusters_knn_conn, adj_knn_conn, first_k_eigenvectors_conn = Spectral_clustering_knn_connectivity(X,len(np.unique(y)),10)
end_conn = time.time()
running_time_conn = end_conn - start_conn

ari_conn = adjusted_rand_score(y, p_ij_knn_conn)
nmi_conn = normalized_mutual_info_score(y, p_ij_knn_conn)

In [7]:
start_dist = time.time()
x_knn_dist, p_ij_knn_dist, distortion_knn_dist, num_clusters_knn_dist, adj_knn_dist, first_k_eigenvectors_dist = Spectral_clustering_knn_distance(X,len(np.unique(y)),10)
end_dist = time.time()
running_time_dist = end_dist - start_dist

ari_dist = adjusted_rand_score(y, p_ij_knn_dist)
nmi_dist = normalized_mutual_info_score(y, p_ij_knn_dist)

In [10]:
print("-----------------------------------------------------------------------------------------------")
print("Spectral Clustering using k-eigenvectors | Mode = connectivity")
print("-----------------------------------------------------------------------------------------------")
print("Adjusted Rand Index (ARI):", ari_conn)
print("Normalized Mutual Information (NMI):", nmi_conn)
print("Running time for normal Spectral Clustering (k-nearest neighbors) is: ", running_time_conn)
print("\n")
print("-----------------------------------------------------------------------------------------------")
print("Spectral Clustering using k-eigenvectors | Mode = distance")
print("-----------------------------------------------------------------------------------------------")
print("Adjusted Rand Index (ARI):", ari_dist)
print("Normalized Mutual Information (NMI):", nmi_dist)
print("Running time for normal Spectral Clustering (k-nearest neighbors) is: ", running_time_dist)
print("-----------------------------------------------------------------------------------------------")

-----------------------------------------------------------------------------------------------
Spectral Clustering using k-eigenvectors | Mode = connectivity
-----------------------------------------------------------------------------------------------
Adjusted Rand Index (ARI): 0.49861484933924655
Normalized Mutual Information (NMI): 0.6971583494219175
Running time for normal Spectral Clustering (k-nearest neighbors) is:  1189.0399975776672


-----------------------------------------------------------------------------------------------
Spectral Clustering using k-eigenvectors | Mode = distance
-----------------------------------------------------------------------------------------------
Adjusted Rand Index (ARI): 0.33226710176178137
Normalized Mutual Information (NMI): 0.5473060829884699
Running time for normal Spectral Clustering (k-nearest neighbors) is:  1078.832494020462
-----------------------------------------------------------------------------------------------
